In [3]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from tifffile import imread, imwrite
import numpy as np
import pandas as pd
from skimage.draw import polygon
from pathlib import Path
from loguru import logger
from IPython.display import clear_output

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.core_cutting.channel_scanner import discover_channels, build_transfer_map
from multiplex_pipeline.core_cutting.controller import CorePreparationController
from multiplex_pipeline.core_cutting.file_io import GlobusFileStrategy
from multiplex_pipeline.utils.globus_utils import GlobusConfig, create_globus_tc
from multiplex_pipeline.utils.file_utils import GlobusPathConverter

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [5]:
# define pathways
settings_path = r'C:\sdata_NSR7649\analysis_settings_NSR7649.yaml'

globus_config_path = r'D:\globus_config\globus_config.yaml'

### Load analysis settings

In [6]:
# load analysis configuration
settings = load_analysis_settings(settings_path)
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/NSR7649/NSR7649_Final', analysis_name='NSR7649_Analysis', local_analysis_dir='C:/sdata_NSR7649', remote_analysis_dir='/ix1/kkedziora', log_dir=None), core_detection=CoreDetectionSettings(detection_image='NSR7649_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=None, ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanup_enabled=True), additional_elements=[NormalizeStep(category='image_transformer', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_transformer', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(low=1.0, high=99.8)), Insta

### Define the logger

In [7]:
log_file = settings.log_dir_path / f"cores_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Set up Globus

In [8]:
# get globus config
gc = GlobusConfig.from_config_files(globus_config_path, from_collection = 'r_collection_id', to_collection = 'cbi_collection_id')
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [9]:
# if Windows paths change to Globus
image_path = settings.general.image_dir
if ":/" in settings.general.image_dir or ":\\" in settings.general.image_dir:
    conv = GlobusPathConverter(layout="single_drive")
    image_path = conv.windows_to_globus(image_path)

image_path

'/CellDive/NSR7649/NSR7649_Final'

In [14]:
channel_map = discover_channels(image_path,
                                include_channels=settings.core_cutting.include_channels,
                                exclude_channels=settings.core_cutting.exclude_channels,
                                use_markers=settings.core_cutting.use_markers,
                                ignore_markers=settings.core_cutting.ignore_markers,
                                gc=gc)

2025-11-06 13:46:57.037 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:80 - Discovered 19 channels:
2025-11-06 13:46:57.038 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_AP2B <- /CellDive/NSR7649/NSR7649_Final/NSR7649_1.0.4_R000_Cy5_AP2B-AF647_FINAL_AFR_F.ome.tif
2025-11-06 13:46:57.038 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_DAPI <- /CellDive/NSR7649/NSR7649_Final/NSR7649_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-11-06 13:46:57.038 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_FOXA1 <- /CellDive/NSR7649/NSR7649_Final/NSR7649_1.0.4_R000_Cy3_FOXA1-AF555_FINAL_AFR_F.ome.tif
2025-11-06 13:46:57.038 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 002_DAPI <- /CellDive/NSR7649/NSR7649_Final/NSR7649_2.0.4_R000_DAPI__FINAL_F.ome.tif
2025-11-06 13:46:57.038 | INFO     | m

In [10]:
df_path = settings['core_info_file_path'].with_suffix('.pkl')

df = pd.read_pickle(df_path)
df.head()

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,832.0,6528.0,7552.0,13120.0,rectangle,"[[832.0, 13120.0], [6528.0, 13120.0], [6528.0,..."
1,Core_001,256.0,6144.0,27520.0,33408.0,rectangle,"[[256.0, 33408.0], [6144.0, 33408.0], [6144.0,..."
2,Core_002,384.0,6208.0,34496.0,40384.0,rectangle,"[[384.0, 40384.0], [6208.0, 40384.0], [6208.0,..."
3,Core_003,768.0,6336.0,41408.0,46976.0,rectangle,"[[768.0, 46976.0], [6336.0, 46976.0], [6336.0,..."
4,Core_004,1088.0,6272.0,14336.0,19840.0,rectangle,"[[1088.0, 19840.0], [6272.0, 19840.0], [6272.0..."


In [16]:
# build transfer map
transfer_cache_dir = settings.temp_dir
transfer_map = build_transfer_map(channel_map, transfer_cache_dir)

# build a dict for transfered images
image_paths = {
    ch: str(Path(transfer_cache_dir) / Path(remote).name)
    for ch, (remote, _) in transfer_map.items()
}
image_paths

{'AP2B': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_4.0.4_R000_Cy5_AP2B-AF647_FINAL_AFR_F.ome.tif',
 'DAPI': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'FOXA1': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_1.0.4_R000_Cy3_FOXA1-AF555_FINAL_AFR_F.ome.tif',
 'ER': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_2.0.4_R000_Cy3_ER-AF555_FINAL_AFR_F.ome.tif',
 'GATA3': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_2.0.4_R000_Cy5_GATA3-AF647_FINAL_AFR_F.ome.tif',
 'AR': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_3.0.4_R000_Cy3_AR-AF555_FINAL_AFR_F.ome.tif',
 'HER2': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_3.0.4_R000_Cy5_HER2-AF647_FINAL_AFR_F.ome.tif',
 'ECad': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_4.0.4_R000_Cy3_ECad-AF555_FINAL_AFR_F.ome.tif',
 'CD45': 'C:\\sdata_NSR7649\\NSR7649_Analysis\\temp\\NSR7649_5.0.4_R000_Cy5_CD45-AF647_FINAL_AFR_F.ome.tif',
 'CK818': 'C:\\sdata_NSR7649\\NSR764

In [12]:
transfer_map

{'DAPI': ('/CellDive/BLCA-2/BLCA-2_Final/BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif',
  '/C/BLCA-2_Analysis_todel/temp/BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif')}

In [15]:
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/NSR7649/NSR7649_Final', analysis_name='NSR7649_Analysis', local_analysis_dir='C:/sdata_NSR7649', remote_analysis_dir='/ix1/kkedziora', log_dir=None), core_detection=CoreDetectionSettings(detection_image='NSR7649_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=None, ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanup_enabled=True), additional_elements=[NormalizeStep(category='image_transformer', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_transformer', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(low=1.0, high=99.8)), Insta

In [ ]:
strategy = GlobusFileStrategy(tc=tc, transfer_map=transfer_map, gc=gc, cleanup_enabled = True) # submits transfers when initialized, should it stay like this?

controller = CorePreparationController(
    metadata_df = df, # df defines which cores to process
    image_paths = channel_map, # defines which channels to use
    temp_dir = settings.cores_dir_tif_path,
    output_dir = settings.cores_dir_output_path,
    file_strategy = strategy,
    margin = settings.core_cutting.margin,
    mask_value = settings.core_cutting.mask_value,
    max_pyramid_levels = settings.sdata_storage.max_pyramid_level,
    chunk_size = settings.sdata_storage.chunk_size,
    downscale = settings.sdata_storage.chunk_size,
)

controller.run()

2025-10-15 14:56:12.519 | INFO     | multiplex_pipeline.core_cutting.file_io:submit_all_transfers:91 - Submitted transfer for DAPI to /C/BLCA-2_Analysis_todel/temp/BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif (task_id=9d4b7100-a9f8-11f0-812b-0e092d85c59b)
2025-10-15 14:56:12.522 | INFO     | multiplex_pipeline.core_cutting.controller:run:117 - Starting controller run loop...
2025-10-15 14:57:23.006 | INFO     | multiplex_pipeline.core_cutting.file_io:is_channel_ready:16 - Transfer for DAPI complete: /C/BLCA-2_Analysis_todel/temp/BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-10-15 14:57:24.329 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:93 - Cut and saved core Core_001, channel DAPI
2025-10-15 14:57:24.332 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:100 - Closed file handle for channel DAPI
2025-10-15 14:57:25.102 | INFO     | multiplex_pipeline.core_cutting.file_io:cleanup:212 - Cleaned up file: C:\BLCA-2_Analysis_todel\temp\BLCA-2_1.0.4_R000_D